# Project steps explained
In this notebook we will explain the different steps f our approach

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# loading first table
bookmarks=pd.read_csv("/content/drive/My Drive/Colab Notebooks/bookmarks.csv")
bookmarks.time = 1	# we set this column to one, because we are gonna use it later in calculating interest

# loading second table
favorites=pd.read_csv("/content/drive/My Drive/Colab Notebooks/favorites.csv")
favorites.added_date = 5 # we set this column to 5, because we are gonna use it later in calculating interest

#loading the third table
ratings=pd.read_csv("/content/drive/My Drive/Colab Notebooks/ratings.csv")
ratings=ratings.drop(columns=['time'])	# we drop this column, cause it has no use to us

In [ ]:
# with this, we merge the two tables, adding the scores +5 of favorite to the adgecent lines
bookmarks = bookmarks.merge(favorites,how='left', 
			left_on=['id_profile','id_asset'], right_on=['id_profile','id_asset'])

# with this, we merge the two tables, adding the ratings of movies to the adgecent lines
bookmarks = bookmarks.merge(ratings,how='left', 
			left_on=['id_profile','id_asset'], right_on=['id_profile','id_asset'])

# filling al he na values with 0, to facilitate the calculation
bookmarks=bookmarks.fillna(0)
# renaming so it would make beter sense
bookmarks=bookmarks.rename(columns={"time": "m_ui", "added_date": "f_ui","score":"n_ui"})

In [ ]:
# calculating interest
bookmarks.m_ui += bookmarks.f_ui + bookmarks.n_ui 

In [ ]:
bookmarks = bookmarks.drop(columns=['f_ui',	'n_ui']).rename(columns={"m_ui": "r_ui"})
bookmarks

In [ ]:
bookmarks.to_csv(path_or_buf="/content/drive/My Drive/Colab Notebooks/D.csv"
,index=False)

In [ ]:
D = pd.read_csv("/content/drive/My Drive/Colab Notebooks/D.csv")

In [ ]:
# loading indexes
train_idx = np.load("/content/drive/My Drive/Colab Notebooks/bookmarks_idx_train.npy")
test_idx = np.load("/content/drive/My Drive/Colab Notebooks/bookmarks_idx_test.npy")


In [ ]:
D_train = D.loc[train_idx]
D_train.reset_index(inplace=True)
D_train.to_csv(path_or_buf="/content/drive/My Drive/Colab Notebooks/D_train.csv"
,index=False)

In [ ]:
D_test = D.loc[test_idx]
D_test.reset_index(inplace=True)
D_test.to_csv(path_or_buf="/content/drive/My Drive/Colab Notebooks/D_test.csv"
,index=False)

In [ ]:
# Reading the trainig data set 
import numpy as np
import pandas as pd
D_train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/D_train.csv")
D_train= D_train.drop(columns=['index'])

# Baseline Estimates

In [ ]:
user_set = set(D_train.id_profile)
movies_set = set(D_train.id_asset)

In [ ]:
b_u_pd = pd.DataFrame(user_set, columns=['id_profile'])
b_u_pd['bu'] = 0.0 
print(b_u_pd)


b_i_pd = pd.DataFrame(movies_set,
                   columns=['id_asset'])
b_i_pd['bi'] = 0.0
print(b_i_pd)

In [ ]:
mu = np.mean(D_train.r_ui)
lr = 0.0001
reg_coeif = 0.02
epochs = 5

In [ ]:
D_train['err'] = 0.0
D_train = D_train.merge(b_u_pd,how='left', left_on=['id_profile'], right_on=['id_profile'])
D_train = D_train.merge(b_i_pd,how='left', left_on=['id_asset'], right_on=['id_asset'])
D_train

In [ ]:
mse_track = list()
for i in range(0, 15):
  
  D_train['err'] = D_train['r_ui'] - D_train['bu'] - D_train['bi'] - mu

  D_train['bu'] = (-2 * D_train['err'] + 2 * reg_coeif * D_train['bu'])
  D_train['bi'] =  (-2 *D_train['err'] + 2 * reg_coeif * D_train['bi'])

  b_u_pd['bu'] -= 1/len(D_train) * lr * D_train.groupby(["id_profile"]).bu.sum().reset_index()['bu']
  b_i_pd['bi'] -= 1/len(D_train) * lr * D_train.groupby(["id_asset"]).bi.sum().reset_index()['bi']

  D_train= D_train.drop(columns=['bu','bi'])

  D_train = D_train.merge(b_u_pd,how='left', left_on=['id_profile'], right_on=['id_profile'])
  D_train = D_train.merge(b_i_pd,how='left', left_on=['id_asset'], right_on=['id_asset'])

  mse_track.append(1/len(D_train) * ((D_train['err']**2).sum()))
  print(mse_track)


In [ ]:
b_u_pd.to_csv(path_or_buf="/content/drive/My Drive/Colab Notebooks/baseline_bu.csv"
,index=False)
b_i_pd.to_csv(path_or_buf="/content/drive/My Drive/Colab Notebooks/baseline_bi.csv"
,index=False)

In [ ]:
# loading test set
D_test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/D_test.csv")
b_u_pd = pd.read_csv("/content/drive/My Drive/Colab Notebooks/baseline_bu.csv")	#loading weights
b_i_pd = pd.read_csv("/content/drive/My Drive/Colab Notebooks/baseline_bi.csv")	#loading weights

# calculating mu, setting up error column, associating weights to lines
mu = np.mean(D_test.r_ui)
#initiali
D_test['err'] = 0.0
D_test = D_test.merge(b_u_pd,how='left', left_on=['id_profile'], right_on=['id_profile'])
D_test = D_test.merge(b_i_pd,how='left', left_on=['id_asset'], right_on=['id_asset'])
# calculating the error for all the 22M entries
D_test['err'] = D_test['r_ui'] - D_test['bu'] - D_test['bi'] - mu
# MSE: 0.55
print(1/len(D_test) * ((D_test['err']**2).sum()))

# SVD

In [ ]:
import numpy as np
import pandas as pd
D_train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/D_train.csv")

#ndiroha oumba3dddaaa 
D_train= D_train.drop(columns=['index'])
print(D_train)

In [ ]:
# prepocesing of D_train
mu = np.mean(D_train.r_ui)
lr = 0.00001
reg_coeif = 0.02
epochs = 5
factor = 5

user_set = set(D_train.id_profile)
movies_set = set(D_train.id_asset)

b_u_pd = pd.DataFrame(user_set, columns=['id_profile'])
b_u_pd['bu'] = 0.0 
b_i_pd = pd.DataFrame(movies_set,columns=['id_asset'])
b_i_pd['bi'] = 0.0

#initialisation de PU
p_u_pd = pd.DataFrame(user_set,columns=['id_profile'])
p_u_pd['pu'] = p_u_pd.apply(lambda x: np.random.rand(factor).tolist(), axis=1)

#initialisation de Qi
q_i_pd = pd.DataFrame(movies_set,columns=['id_asset'])
q_i_pd['qi'] = q_i_pd.apply(lambda x: np.random.rand(factor).tolist(), axis=1)



In [ ]:
def dot_pu_qi(x, y):
    return np.dot(np.asarray(x), np.asarray(y))

def calculate_delta(x, y, z):
    return ( lr * (-2 * z * np.asarray(x) + 2 * reg_coeif * np.asarray(y))).tolist()

from functools import reduce
def test_sum(series):
  return reduce(lambda x, y: (np.asarray(x) + np.asarray(y)).tolist(), series)

def sub_vec(x, y):
    return (np.asarray(x) - np.asarray(y)).tolist()

In [ ]:
batch_size = 2000000
n_batchs = len(D_train)/batch_size
mse_track = list()
#for i in range(epochs):
for j in range(1):
  Batch = D_train.loc[j*batch_size:(j+1)*batch_size-1]

  # adding columns needed for calculating the error
  Batch['err'] = 0.0
  Batch = Batch.merge(b_u_pd,how='left', left_on=['id_profile'], right_on=['id_profile'])
  Batch = Batch.merge(b_i_pd,how='left', left_on=['id_asset'], right_on=['id_asset'])
  Batch = Batch.merge(p_u_pd,how='left', left_on=['id_profile'], right_on=['id_profile'])
  Batch = Batch.merge(q_i_pd,how='left', left_on=['id_asset'], right_on=['id_asset'])

  # calculating the error
  Batch['err'] = Batch['r_ui'] - Batch['bu'] - Batch['bi'] - mu - Batch[["pu", "qi"]].apply(lambda x : dot_pu_qi(*x), axis=1)

  # calculating the error in regard to the parameters
  Batch['bu'] = 1/200000 * lr * (-2 * Batch['err'] + 2 * reg_coeif * Batch['bu'])
  Batch['bi'] = 1/200000 * lr * (-2 *Batch['err'] + 2 * reg_coeif * Batch['bi'])
  Batch["delta_pu"] = Batch[["qi", "pu", "err"]].apply(lambda x : calculate_delta(*x), axis=1)
  Batch["delta_qi"] = Batch[["pu", "qi", "err"]].apply(lambda x : calculate_delta(*x), axis=1)

  # updating the b_u weights, todoso we create a dataframe containing the error than substract it 
  delta_bu = (Batch.groupby(["id_profile"]).bu.sum().reset_index()).rename(columns={"bu": "bu_delta"})
  b_u_pd = (b_u_pd.merge(delta_bu,how='left', left_on=['id_profile'], right_on=['id_profile'])).fillna(0)
  b_u_pd['bu'] -= b_u_pd['bu_delta']
  b_u_pd= b_u_pd.drop(columns=['bu_delta'])

  # updating the b_i weights, todoso we create a dataframe containing the error than substract it 
  delta_bi = (Batch.groupby(["id_asset"]).bi.sum().reset_index()).rename(columns={"bi": "bi_delta"})
  b_i_pd = (b_i_pd.merge(delta_bi,how='left', left_on=['id_asset'], right_on=['id_asset'])).fillna(0)
  b_i_pd['bi'] -= b_i_pd['bi_delta']
  b_i_pd= b_i_pd.drop(columns=['bi_delta'])

  # updating the p_u weights, todoso we create a dataframe containing the error than substract it 
  delta_pu = Batch[['id_profile', 'delta_pu']].groupby('id_profile').agg({'delta_pu': [test_sum]})
  delta_pu.columns = delta_pu.columns.droplevel(1)
  p_u_pd = (p_u_pd.merge(delta_pu,how='left', left_on=['id_profile'], right_on=['id_profile'])).fillna(0)
  p_u_pd["pu"] = p_u_pd[["pu", "delta_pu"]].apply(lambda x : sub_vec(*x), axis=1)
  p_u_pd= p_u_pd.drop(columns=['delta_pu'])

  # updating the q_i weights, todoso we create a dataframe containing the error than substract it 
  delta_qi = Batch[['id_asset', 'delta_qi']].groupby('id_asset').agg({'delta_qi': [test_sum]})
  delta_qi.columns = delta_qi.columns.droplevel(1)
  q_i_pd = (q_i_pd.merge(delta_qi,how='left', left_on=['id_asset'], right_on=['id_asset'])).fillna(0)
  q_i_pd["qi"] = q_i_pd[["qi", "delta_qi"]].apply(lambda x : sub_vec(*x), axis=1)
  q_i_pd= q_i_pd.drop(columns=['delta_qi'])

  
  #vérfier pour les 400 derniers individus 
  ############################
  '''b_u_pd['bu'] -= 1/len(D_train) * D_train.groupby(["id_profile"]).bu.sum().reset_index()['bu']
  b_i_pd['bi'] -= 1/len(D_train) * D_train.groupby(["id_asset"]).bi.sum().reset_index()['bi']'''
  #D_train= D_train.drop(columns=['bu','bi', 'pu', 'qi', ])


  
  mse_track.append(1/len(Batch) * ((Batch['err']**2).sum()))
  print(mse_track)

In [ ]:
b_u_pd.to_csv(path_or_buf="/content/drive/My Drive/Colab Notebooks/SVD_bu.csv" ,index=False)
b_i_pd.to_csv(path_or_buf="/content/drive/My Drive/Colab Notebooks/SVD_bi.csv" ,index=False)
p_u_pd.to_csv(path_or_buf="/content/drive/My Drive/Colab Notebooks/SVD_pu.csv" ,index=False)
q_i_pd.to_csv(path_or_buf="/content/drive/My Drive/Colab Notebooks/SVD_qi.csv" ,index=False)

In [ ]:
# loading test set
D_test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/D_test.csv")
b_u_pd = pd.read_csv("/content/drive/My Drive/Colab Notebooks/SVD_bu.csv")	#loading weights
b_i_pd = pd.read_csv("/content/drive/My Drive/Colab Notebooks/SVD_bi.csv")	#loading weights
p_u_pd = pd.read_csv("/content/drive/My Drive/Colab Notebooks/SVD_bu.csv")	#loading weights
q_i_pd = pd.read_csv("/content/drive/My Drive/Colab Notebooks/SVD_bi.csv")	#loading weights

# calculating mu, setting up error column, associating weights to lines
mu = np.mean(D_test.r_ui)
#initiali
D_test = D_test.merge(b_u_pd,how='left', left_on=['id_profile'], right_on=['id_profile'])
D_test = D_test.merge(b_i_pd,how='left', left_on=['id_asset'], right_on=['id_asset'])
D_test = D_test.merge(p_u_pd,how='left', left_on=['id_profile'], right_on=['id_profile'])
D_test = D_test.merge(q_i_pd,how='left', left_on=['id_asset'], right_on=['id_asset'])

def dot_pu_qi(x, y):
    return np.dot(np.asarray(x), np.asarray(y))

# calculating the error
D_test['err'] = D_test['r_ui'] - D_test['bu'] - D_test['bi'] - mu - D_test[["pu", "qi"]].apply(lambda x : dot_pu_qi(*x), axis=1)
# MSE: 2.11
print(1/len(D_test) * ((D_test['err']**2).sum()))